In [ ]:
# Plan-and-Solve

In [ ]:
# 系统prompt样例
# 一个标准的系统提示词的必要条件：
# 角色设定： “顶级的AI规划专家”，激发模型的专业能力。
# 任务描述： 清晰地定义了“分解问题”的目标。
# 格式约束： 强制要求输出为一个 Python 列表格式的字符串，这极大地简化了后续代码的解析工作，使其比解析自然语言更稳定、更可靠。

PLANNER_PROMPT_TEMPLATE = """
你是一个顶级的AI规划专家。你的任务是将用户提出的复杂问题分解成一个由多个简单步骤组成的行动计划。
请确保计划中的每个步骤都是一个独立的、可执行的子任务，并且严格按照逻辑顺序排列。
你的输出必须是一个Python列表，其中每个元素都是一个描述子任务的字符串。

问题: {question}

请严格按照以下格式输出你的计划,```python与```作为前后缀是必要的:
```python
["步骤1", "步骤2", "步骤3", ...]
```
"""


In [ ]:
# 将提示词封装成一个Planner类

from helloLLM import HelloAgentsLLM
import ast

class Planner:
    """
    import ast\n
    根据用户问题生成一个行动计划
    """
    def __init__(self, llm_client):
        self.llm_client = llm_client
        
    def plan(self, question: str) -> list[str]:
        """
        根据用户问题生成一个行动计划
        """
        prompt = PLANNER_PROMPT_TEMPLATE.format(question=question)

        # 为了生成计划，构建一个简单的消息列表
        messages = [{"role": "user", "content": prompt}]

        print("--- 正在生成计划 ---")
        # 使用流式输出来获取完整的计划
        response_text = self.llm_client.think(messages=messages) or ""

        print(f"计划已生成：\n{response_text}")

        # 解析LLM输出的列表字符串
        try:
            # 找到```python和```之间的内容
            plan_str = response_text.split("```python")[1].split("```")[0].strip()
            # 使用ast.literal_eval来安全地执行字符串，将其转换为python列表
            plan = ast.literal_eval(plan_str)
            return plan if isinstance(plan, list) else []
        
        except (ValueError, SyntaxError, IndexError) as e:
            print(f"解析计划时出错：{e}")
            print(f"原始响应：{response_text}")
            return []
        
        except Exception as e:
            print(f"解析计划时发生未知错误：{e}")
            return []